In [1]:
from PIL import Image
import os
import pandas as pd
import numpy as np

# Ingest raw data contained in raw_data.zip

In [3]:
sgsdir = 'SGS'
dcgandir = '256_dcgan_cp_150/256_dcgan_cp_150'
diffusiondir = '256_sim_cp_10_100.csv'


coord = pd.read_csv('coords_256.csv').values

In [4]:
#SGS - 50 realizations
# sgs = np.empty((len(os.listdir(sgsdir)), 256, 256))

# for i, name in enumerate(os.listdir(sgsdir)):
#     sgs[i] = pd.read_parquet(sgsdir+"/"+name).values.reshape(300,300)[:256, :256]

In [5]:
def ImgData(filename):

    imgs = np.empty((len(os.listdir(filename)), 256**2))

    for i, name in enumerate(os.listdir(filename)):
        path = os.path.join(filename, name)
        image = Image.open(path).convert('L')
        image = np.array(image, dtype=np.float32) / 255.0

        imgs[i] = image.reshape(256**2)

    return imgs

sgs = ImgData('SGS')
sgs = sgs.reshape((sgs.shape[0], 256, 256))
sgs.shape

(100, 256, 256)

In [6]:
sgs = (sgs + 1)/2

In [7]:
coord.shape, sgs.shape

((65536, 2), (100, 256, 256))

In [8]:
x_uniq = np.unique(coord[:, 0])
y_uniq = np.unique(coord[:, 1])

xx, yy = np.meshgrid(x_uniq, y_uniq)

xx.shape, yy.shape

((256, 256), (256, 256))

In [9]:
x_uniq[1]-x_uniq[0], y_uniq[1]-y_uniq[0]

(500.0, 500.0)

## DCGAN

In [10]:
# DCGAN - 100 samples
dcgan = np.empty((len(os.listdir(dcgandir)), 256, 256))

for i, name in enumerate(os.listdir(dcgandir)):
        path = os.path.join(dcgandir, name)
        image = Image.open(path).convert('L')
        image_normalized = np.array(image, dtype=np.float32) / 255.0
        
        
        dcgan[i] = image_normalized

dcgan.shape

(100, 256, 256)

## Diffusion

In [11]:
# Diffusion - 50 samples (randomly sampled from 100 original -- too big to push to repo!)

diffusion = pd.read_csv(diffusiondir, header = None)
diffusion = diffusion.to_numpy().reshape((100, 256, 256))
diffusion.shape

(100, 256, 256)

In [12]:
dmin = -665.37
dmax = 636.33

def rescale_range(x, a, b):
    return a+((x-np.nanmin(x))*(b-a))/(np.nanmax(x)-np.nanmin(x))

sgs = rescale_range(sgs, dmin, dmax)
diffusion = rescale_range(diffusion, dmin, dmax)
dcgan = rescale_range(dcgan, dmin, dmax)

In [14]:
np.savez_compressed('data.npz', sgs=sgs, diffusion=diffusion, dcgan=dcgan, xx=xx, yy=yy)